<a href="https://colab.research.google.com/github/PythonJulianPrieto/CienciaDeDatos/blob/main/quijote.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PySpark instalación

Pasos para iniciar en un proyecto de PySpark

1) Actualizar el sistema operativo de ubuntu en Google Colab.


In [ ]:
#!ls -la

#!java -version

#!mkdir spark

#Actualizamos el sistema operativo, antes de instalar Java y de instalar Spark
#Requiere el root
#!sudo 

In [ ]:
#Descargar el paquete de Spark con la version de Hadoop

!wget https://downloads.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz

--2021-12-28 16:03:19--  https://downloads.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 88.99.95.219, 2a01:4f8:10a:201a::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 300965906 (287M) [application/x-gzip]
Saving to: ‘spark-3.2.0-bin-hadoop3.2.tgz.1’

spark-3.2.0-bin-had 100%[===================>] 287.02M  10.2MB/s    in 30s     

2021-12-28 16:03:50 (9.56 MB/s) - ‘spark-3.2.0-bin-hadoop3.2.tgz.1’ saved [300965906/300965906]



In [ ]:
#Descomprimimos la version de Spark y Hadoop

!tar -xvf /content/spark-3.2.0-bin-hadoop3.2.tgz

Configurar la variable de ambiente de Spark.

In [ ]:
import os

os.environ['SPARK_HOME'] = '/content/spark-3.2.0-bin-hadoop3.2'

#INSTALAR SPARK DENTRO DE PYTHON

In [ ]:
#Cargar install 
!pip install -q findspark

In [ ]:
#Realizaremos el import de findspark dentro de la sesión actual 

import findspark

findspark.init()

El SparkContext me permite definir las variables de entorno de la sesión

SparkConf permite configurar la sesión para interactuar, debe definirce el nombre de la aplicación y cual es el master del cluster. si va a usar una maquina local si un cluster o esta ejecuntado el python en una maquina local debe escribir **Local[*]**

In [ ]:
from pyspark import SparkConf, SparkContext



In [ ]:
#Lancemos una aplicacion en Hadoop, utilizando spark llamada taller y el nodo maestro a usar va a ser la maquina local
conf = SparkConf().setAppName("Quijote").setMaster("local[*]")
sc = SparkContext(conf=conf)

#**Descargar el libro de don quijote**

In [ ]:
!wget https://raw.githubusercontent.com/adiacla/bigdata/master/donQuijote.txt

--2021-12-28 16:05:37--  https://raw.githubusercontent.com/adiacla/bigdata/master/donQuijote.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2198927 (2.1M) [text/plain]
Saving to: ‘donQuijote.txt’

donQuijote.txt      100%[===================>]   2.10M  --.-KB/s    in 0.08s   

2021-12-28 16:05:38 (27.1 MB/s) - ‘donQuijote.txt’ saved [2198927/2198927]



In [ ]:
libro = sc.textFile('donQuijote.txt')

In [ ]:
libro.take(10)

['The Project Gutenberg EBook of Don Quijote, by Miguel de Cervantes Saavedra',
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included',
 'with this eBook or online at www.gutenberg.net',
 '',
 '',
 'Title: Don Quijote',
 '']

In [ ]:
libro.count()

37861

In [ ]:
#Saber cuantas veces se repite la palabra quijote
numeroQuijote = libro.filter(lambda x:"Quijote" in x)
numeroQuijote.take(10)

['The Project Gutenberg EBook of Don Quijote, by Miguel de Cervantes Saavedra',
 'Title: Don Quijote',
 'El ingenioso hidalgo don Quijote de la Mancha',
 'vulgo, he determinado de sacar a luz al Ingenioso hidalgo don Quijote de la',
 'Don Quijote, no quiero irme con la corriente del uso, ni suplicarte, casi',
 'había de hacer a la historia de don Quijote, y que me tenía de suerte que',
 'yo determino que el señor don Quijote se quede sepultado en sus archivos en',
 'famoso don Quijote, luz y espejo de toda la caballería andante.',
 'tan sincera y tan sin revueltas la historia del famoso don Quijote de la',
 '   tus proezas envidio, ¡oh gran Quijote!']

In [ ]:
numeroQuijote.count()

2159

In [ ]:
separadosRDD = libro.flatMap(lambda x:x.split(" "))
separadosRDD.take(10)

['The',
 'Project',
 'Gutenberg',
 'EBook',
 'of',
 'Don',
 'Quijote,',
 'by',
 'Miguel',
 'de']

In [ ]:
qMayusculas = separadosRDD.filter(lambda x:x.startswith("d"))
qMayusculas.take(10)

['de', 'de', 'don', 'de', 'de', 'de', 'del', 'de', 'doy', 'dél']

In [ ]:
#La siguiente funcion lambda me mapea las palabras en pares
parPalabras = separadosRDD.map(lambda x:(x,1))
parPalabras.take(10)

[('The', 1),
 ('Project', 1),
 ('Gutenberg', 1),
 ('EBook', 1),
 ('of', 1),
 ('Don', 1),
 ('Quijote,', 1),
 ('by', 1),
 ('Miguel', 1),
 ('de', 1)]

In [ ]:
cantidadPalabra = parPalabras.reduceByKey(lambda x,y: x+y)
cantidadPalabra.take(10)

[('The', 12),
 ('Project', 80),
 ('EBook', 1),
 ('of', 107),
 ('Don', 96),
 ('Quijote,', 531),
 ('Miguel', 21),
 ('', 9504),
 ('is', 25),
 ('use', 11)]

In [ ]:
#Salvar un RDD en un archivo en el sistema operativo :D
cantidadPalabra.saveAsTextFile('cantidadPalabraDeDonQuijote')

In [ ]:
!head ./cantidadPalabraDeDonQuijote/part-00001

('Gutenberg', 22)
('by', 23)
('de', 17988)
('Cervantes', 12)
('Saavedra', 3)
('This', 5)
('eBook', 5)
('for', 21)
('the', 157)
('cost', 2)


In [ ]:
#Encontrar dentro de don Quijote, cuales lineas tienen digitos o numeros
digitos = libro.filter(lambda x: any(i.isdigit()for i in x))
digitos.take(10)

['Posting Date: April 27, 2010 [EBook #2000]',
 'Release Date: December, 1999',
 'de 1604 años.',
 'signis Ecclesiae, cap. 10, alentando ánimos marchitos y espíritus',
 'Madrid, a 17 de marzo de 1615.',
 'habéis señalado, 23 días ha que salimos de nuestro pueblo: contad, Sancho,',
 'veinte de julio de 1614.',
 'como se espera de vuestro entendimiento. Deste lugar, a 16 de agosto, a las',
 '***** This file should be named 2000-8.txt or 2000-8.zip *****',
 '        http://www.gutenberg.org/2/0/0/2000/']

In [ ]:
libro.filter(lambda x: any(i.isdigit()for i in x)).count()

67

In [ ]:
libro.filter(lambda x: not any(i.isdigit()for i in x)).count()

37794

In [ ]:
libro.filter(lambda x: not any(i.isdigit()for i in x)).take(10)

['The Project Gutenberg EBook of Don Quijote, by Miguel de Cervantes Saavedra',
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included',
 'with this eBook or online at www.gutenberg.net',
 '',
 '',
 'Title: Don Quijote',
 '']